## 實作案例: 請先切換kernel 於您新安裝的 kernel環境
## <span style="color:red">Change to kernel:   Image_XXXXXXXXXXl</span>.

https://python.langchain.com.cn/docs/get_started/quickstart

In [ ]:
# 初始環境設定
import os
from pathlib import Path
HOME = str(Path.home())
Add_Binarry_Path=HOME+'/.local/bin:/usr/ubuntu_bin'
os.environ['PATH']=os.environ['PATH']+':'+Add_Binarry_Path

In [ ]:
# LOAD LIBRARY
from torch import cuda, bfloat16, float16
from langchain.chains import LLMChain, SimpleSequentialChain, RetrievalQA
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, GenerationConfig, pipeline, TextStreamer

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

MODEL_ID = "/work/u00cjz00/slurm_jobs/github/models/Llama-2-7B-Chat-GPTQ"
model = AutoModelForCausalLM.from_pretrained(MODEL_ID, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

llm = HuggingFacePipeline(pipeline=pipe, model_kwargs={"temperature": 0})

In [ ]:
llm("Explain the difference between ChatGPT and open source LLMs in a couple of lines.")

In [ ]:
### Prompt Template
prompt_template = '''
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>
{question} [/INST]

'''

prompt = PromptTemplate(
    input_variables=["question"],
    template=prompt_template,
)

In [ ]:
question = "Explain what are Deep Neural Networks in 2-3 sentences"
print(prompt.format(question=question))

In [ ]:
llm(prompt.format(question=question))

In [ ]:
## Create Chain01
from langchain.chains import LLMChain
chain01 = LLMChain(llm=llm, prompt=prompt)
result01 = chain01.run(question=question)
print(result01.strip())

In [ ]:
## Create Chain02
### Prompt Template for summary
prompt_template = "<s>[INST] Use the summary {summary} and give 3 examples of practical applications with 1 sentence explaining each [/INST]"

prompt02 = PromptTemplate(
    input_variables=["summary"],
    template=prompt_template,
)
chain02 = LLMChain(llm=llm, prompt=prompt02)
print(prompt02.format(summary=result01))
result02 = chain02.run(result01)
print(result02.strip())

In [ ]:
## Chaining Chains
## multi_chain
multi_chain = SimpleSequentialChain(chains=[chain01, chain02], verbose=True)
result_mutiple = multi_chain.run(question)
print(result_mutiple.strip())